<a href="https://colab.research.google.com/github/tusker4/Sesac_Saltlux_DeepLearning/blob/main/5_NLP_Transformers_kogpt2_%EB%AC%B8%EC%9E%A5%EC%83%9D%EC%84%B1_%EC%A0%9C%EB%A1%9C%EC%83%B7%EB%9F%AC%EB%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- 주소
  - https://github.com/SKT-AI/KoGPT2

- 목표
  - 오랜시간에 걸쳐서 대량의 데이터로 사전학습된 모델을 가져와서, 문장 생성 서비스를 생성
  - transformers 계열 모델 사용(허깅페이스 플랫폼 기반)

- 사전 크기
  - 51,200자

- 학습
  - 사전 학습된 모델 그대로 사용(제로샷)
  - 향후, 수집한 데이터를 학습시켜서 (파인튜닝 수행) 특정데이터에 특화된 모델을 생성

- 서비스
  - flask 챗봇 서비스에 이식 -> 문장이 완성되서 한번에 출력 혹은 단어가 생성되면 바로바로 전송해서 문장이 완성되게 하는 방식
  - aws 적용

## 트렌스포머 설치

In [ ]:
!pip install transformers

#  kogpt2-base-v2-로드
- 레퍼지토리 명  
  - URL : https://huggingface.co/skt/kogpt2-base-v2
  - **skt/kogpt2-base-v2**
- 2개 모델 로드
  - 토크나이저
    - 사전에 텍스트->사전화->백터화->패딩->임베딩
    - 텍스트 -> 인코딩 -> 벡터화 제공
    - 예측(수치) -> 디코딩 -> 텍스트
  -예측모델
    - 입력 -> 텍스트 생성

In [ ]:
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel
# 토치로 만든 모델 -> 텐서플로우로 사용 -> 일부 가중치는 사용 안될수도 있다.(감안)

In [ ]:
repo = 'skt/kogpt2-base-v2' # 허깅페이스에 등록될 사전 학습된 모델의 레파지토리
tokenizer = AutoTokenizer.from_pretrained( repo )
model = TFGPT2LMHeadModel.from_pretrained(repo, from_pt = True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'lm_head.weight']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

# 문장 생성 테스트

In [ ]:
import tensorflow as tf

In [ ]:

# 1. 입력문장 받을 변수
text = None
# 2. 사용자 입력무한대기
while True:
  text = input('작성하고 싶은 글의 초기문장을 넣어주세요\n').strip('')
  # 무한대기종료
  if text == 'z':
    break
  # 비워있는 문자열인 경우
  if not text:
    print('정확하게 입력해주세요')
    continue # 가장 가까운 if문으로

  # 정상처리 시작
  # 1. 택스트 -> 인코딩 -> 문장벡터 획득
  input_vector = tokenizer.encode(text)
  print(input_vector)

  # 2. 이 모델이 원하는 형태로 문장백터로 변환 -> 여기서는 2D-텐서
  input_vector = tf.convert_to_tensor( [input_vector])
  print(input_vector)

  # 3. 모델을 이용하여, 문장 생성 -> 품질?, 문장의 길이?

  output = model.generate(input_vector, max_length=64,  # 생성되는 단어수, 토큰수 최대값
                 use_cache=True, # 캐싱을 활용, 동일 내용에 대한 빠른 응답
                 repetition_penalty=2.0   # 수치는 넣어보면서 체크, 반복적으로 만들어진 문구-> 패널티 적용)
                 )
  # 4.  디코딩 -> 텐서 -> 배열 -> 출력
  # 문장벡터 추출
  output_vector = output.numpy().tolist()[0]
  output_text =   tokenizer.decode(output_vector)
  print('output_text : ', output_text )


  pass

작성하고 싶은 글의 초기문장을 넣어주세요
힘세고 좋은 아침 만약 누가 나를 물어본다면 나는 왈도
[9723, 46777, 10586, 17339, 15359, 22031, 24692, 27515, 7655, 10846, 12102, 27417, 7235]
tf.Tensor(
[[ 9723 46777 10586 17339 15359 22031 24692 27515  7655 10846 12102 27417
   7235]], shape=(1, 13), dtype=int32)
output_text :  힘세고 좋은 아침 만약 누가 나를 물어본다면 나는 왈도왈부할 것이다.
나는 그 말을 듣고 싶지 않다.
그렇지만 내가 대답을 할 수 있는 유일한 방법은, 나의 답변을 듣는 것뿐이다.
내가 질문하는 것은 내 질문에 대한 답을 찾는 일이다.
이것은 나에게 가장 중요한 질문이다.
나의 질문을 통해 우리는 자신의
작성하고 싶은 글의 초기문장을 넣어주세요
z


- 생성된 문장은 학습시 사용한 코퍼스의 영향을 받음
  - 특정분야에 적합한 대답 (->파인튜닝이 필요)
- 문장을 다 모아서 한번에 리턴
  - 사용자는 대기시간이 길어진다 -> 특정토큰이 완성되면 바로전송 -> 기다리지않는다.

In [ ]:
#  문장을 생성하는 응답 방식 조정
import numpy as np
import pandas as pd
import random
import math

In [ ]:
text = '공부를 잘하기 위해서는'
input_vector = tokenizer.encode(text)
# 배열로 차원 추가 -> 2D
input = np.array( [ input_vector ])
print( input )
# 디코딩 확인 -> 형태소 분석기 (한글은 교착어)가 어떻게 적용되어있는지 잠시 확인
# toeknizer.decode(input_vector[:2])
output = model(input)
# print(type(output), output)
# tf.math.top_k() => 활용, 모델을 통해 예측된 다음 단어(토큰)에 대한 상위 x 개를 뽑을 수 있다.
# 항상 1등만 체크하면 => 답은 거의 비슷함, 상위 5개중 랜덤하게 선택(선택방법은 다양) -> 완전 다른 응답이 나올 수 있다.
# tf.math.top_k(output.logits[0, -1]) : 마지막 토큰(다음 단어를 예측하기 위한 문장의 가장 마지막 단어)
# (1, 4, 51200) 에서  (0, 0/1/2) 어텐션등 정보, (0,-1) 이 부분이 다음 단어 예측파트
# 51200 => softmax 확률 세팅 =>  상위 5개만 추출
print(output.logits[0, -1].shape)
top5 = tf.math.top_k(output.logits[0, -1], k=5)
# 입력 문장 기반 내용을 이어가는데 다음 토큰(단어로) 나올 후보들 중 확률이 높은 5개가 추출
for idx in top5.indices.numpy():
  print(idx, tokenizer.decode( idx ) ) # 랜덤 초이스 -> 선택에는 다양한 방법들이 존재함, 랜덤은 그중에 하나

token_id = random.choice(top5.indices.numpy() ) # 주어진 데이터에서 하나 추출
print(token_id, tokenizer.decode(token_id) )





[[21347  9443  9154 11357]]
(51200,)
10635 먼저
23879 무엇보다
12201 우선
11649 어떻게
9745 자신의
11649 어떻게


In [ ]:
# 다음에 나올 토큰을 랜덤하게  처리하여 문장을 생성하는 방식

text = '공부를 잘하기 위해서는'
input_vector = tokenizer.encode(text)

while len(input_vector)<30 : # 제시한 문장 + 새로 만들어지는 문장 => 30토큰이면 종료
  # 1. 모델 예측 => 원문 + 예측토큰(계속 누적)
  # => 보다 완벽한 문장을 유도, 단 랜덤이 성능에 영향을 미침
  # => 최초 입력 문장을 계속 입력 => 연산비용은 상승, 장기기억 분야에서는 도움
  output = model(np.array( [ input_vector ]))
  # 2. 다음 토큰으로 나온 예측값들중 탑 5 추출
  top5 = tf.math.top_k(output.logits[0, -1], k=5)
  # 3. 이중에서 랜덤으로 1개추출
  token_id = random.choice(top5.indices.numpy())
  # 4. 해당 글자를 출력 -> 클라이언트에서 전송(바로 혹은 3-4개씩 묶어서) 혹은 콘솔출력
  print(tokenizer.decode(token_id), end=' ')
  # 5. input_vector에 추가
  input_vector.append(token_id)


자신의 실력을 최대한 끌어올 릴 수밖에 없는 법 이다. 하지만 아무리 열심히 공부하고 열심히 노력하는 것 에도 불구하고, 자신이 하고 싶어 하지 않거나 자신의 적 성이 

In [ ]:
pip list